### Loading of the model

In [1]:
#KERNEL 3.6!

# First, we can import some functions so we can use the model
from cobra.io import read_sbml_model, write_sbml_model
from cobra import Reaction, Metabolite
from cameo.strain_design import pathway_prediction

# Second, we can read the GEM and save it as ‘model’
model = read_sbml_model('data/yeastGEM_het.xml')
# Third, we can show general information about the loaded model
model

Name,M_yeastGEM_v8__46__4__46__2
Memory address,0x01dfdddc2400
Number of metabolites,2745
Number of reactions,4062
Number of groups,90
Objective expression,1.0*r_2111 - 1.0*r_2111_reverse_58b69
Compartments,"cell envelope, cytoplasm, extracellular, mitochondrion, nucleus, peroxisome, endoplasmic reticulum, Golgi, lipid particle, vacuole, endoplasmic reticulum membrane, vacuolar membrane, Golgi membrane, mitochondrial membrane"


### General model information

### FVA analysis

In [2]:
from cobra.flux_analysis.phenotype_phase_plane import production_envelope, add_envelope
import pandas as pd
import matplotlib

In [3]:
#FVA ANALYSIS
#flux variability analysis
import matplotlib.pyplot as plt
from cameo import load_model
from cameo.flux_analysis.analysis import phenotypic_phase_plane
from cameo.visualization.plotting.with_plotly import PlotlyPlotter

plotter = PlotlyPlotter()
from cameo import flux_variability_analysis


In [ ]:
fva_result = flux_variability_analysis(model)
fva_result.data_frame

In [ ]:
#help -> launch classic notebook
from cameo.visualization import plotting
plotter = PlotlyPlotter()
fva_result.plot(plotter,index=fva_result.data_frame.index[4045:])

In [ ]:
fva_result2 = flux_variability_analysis(model,fraction_of_optimum=0.5)
fva_result2.data_frame

In [ ]:
import plotly.graph_objects as go
import matplotlib
import matplotlib.pyplot as plt 
fig = fva_result2.plot(plotter,index=fva_result2.data_frame.index[:25])
fig.write_image("images/fig1.png")


In [ ]:
for reaction in model.reactions.query('lathosterol', 'name'):
    print(reaction.name)

model.metabolites.get_by_id("7-Dehydrocholesterol_c")

### Non-automatic optimization

We should probably try to include some of the knockouts and gen upreagulations that we have found in other papers. That could be a very viable strategy.

### Automatic optimization using OptGene

Note: This section of the code is heavily based around the notebook: 09-Predict-gene-knockout-strategies and this notebook: http://cameo.bio/05-predict-gene-knockout-strategies.html

The OptGene algorithm can be used to find genes which can be knocked out or mutated to increase the production of our target compound. The algorithm uses an approach, which takes a very long time to run. 
To make the program a tiny bit faster, all the essential reactions can be found and excluded from analysis, as they should not be knocked out in any case. Knocking out essential genes would stop growth.

In [ ]:
# In this block, we are finding a list of the essential reactions and the non-essential reactions. 
# By looping over all reactions in the model and seeing whether they can grow, we can sort them into the two different lists

#%%time

essential = []
non_essential = []

for reaction in model.reactions:
    with model:
        reaction.knock_out()
        if model.slim_optimize(error_value=0.) == 0.0:
            essential.append(reaction)
        else:
            non_essential.append(reaction)

In [ ]:
# Following this, we can find the genes associated with all of the essential reactions

#%%time
ess = []
for i in range (1,len(essential)):
    for gene in list(essential[i].genes):
        ess.append(gene)

In [ ]:
from cameo.strain_design.heuristic.evolutionary_based import OptGene

In [ ]:
from cameo.strain_design import OptGene
optgene = OptGene(model)
result = optgene.run(target=model.reactions.R07215,
                     biomass=model.reactions.r_2111,
                     substrate=model.reactions.r_1714,
                     max_evaluations=5000,
                     plot=False)

In [ ]:
result

As we can be seen from this, we have no optimization targets for the OptGene analysis

### OptKnock

Similar to OptGene, OptKnock is an algorithm that can produce knock-out strains to increase the target production. 

In [ ]:
from cameo.strain_design.deterministic.linear_programming import OptKnock
from cameo.strain_design import OptKnock

In [ ]:
optknock = OptKnock(model, fraction_of_optimum=0.1) #excluded reactions virker ikke

In [ ]:
result = optknock.run(max_knockouts=3, target="R07215", biomass="r_2111")

In [ ]:
result

In [ ]:
result.plot(plotter, height=400)

### Phenotypic Phase Plane analysis
A PPP is a great way to learn more about the model and the trade-offs that are going on under the hood.
Unfortuneatly, the plotting error persists and this section needs more work

In [ ]:
from cameo import phenotypic_phase_plane

In [ ]:
r_7DH_export

In [ ]:
ppp = phenotypic_phase_plane(model,
                             variables=[model.reactions.get_by_id("r_2111")],
                             objective=model.reactions.get_by_id("R07215"))

ppp.plot(plotter,height=400)

### FSEOF analysis
We should be able to use FSEOF for something, but, as of writing, I am not sure how to actually use the information here.

In [ ]:
from cameo.strain_design.deterministic.flux_variability_based import FSEOF

In [ ]:
fseof = FSEOF(model)

In [ ]:
fseofresult=fseof.run(target=model.reactions.R07215)

In [ ]:
from matplotlib import pyplot as plt
fseofresult.data_frame.T.plot()
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()

### More PPP
Here, a PPP analysis was conducted usin slightly different code, but plotting works!

In [ ]:
from cameo.flux_analysis.analysis import phenotypic_phase_plane
from cameo.strain_design.deterministic import DifferentialFVA

In [ ]:
production = phenotypic_phase_plane(model,
                                             variables=[model.reactions.get_by_id("r_2111")],
                                             objective=model.metabolites.get_by_id("7-Dehydrocholesterol_c"))
production.plot(plotter, height=400)




In [ ]:
essentiality = {}
for gene in model.genes:
    with model:
        gene.knock_out()
        essentiality[gene] = model.slim_optimize(error_value=0.)

### More pretty PPP

In [ ]:
import cameo
from cameo import load_model
from cameo.flux_analysis.analysis import phenotypic_phase_plane
from cameo.visualization.plotting.with_plotly import PlotlyPlotter

# options may be passed as arguments here
plotter = PlotlyPlotter() #very important!

production_envelope = phenotypic_phase_plane(model,variables=[model.reactions.r_2111],objective=model.metabolites.get_by_id('7-Dehydrocholesterol_c'))
# notice the required plotter argument!
production_envelope.plot(plotter, height=400)

### Automatic optimization strategies


In [ ]:
from cameo.strain_design import OptGene
optgene = OptGene(model)
result = optgene.run(target=model.reactions.R07215,
                     biomass=model.reactions.r_2111,
                     substrate=model.reactions.r_1714,
                     max_evaluations=5000,
                     plot=False)

In [ ]:
result

In [ ]:
from cameo.strain_design import OptKnock
optknock = OptKnock(model, fraction_of_optimum=0.1)
result1 = optknock.run(max_knockouts=3, target="R07215", biomass="r_2111")


In [ ]:
result1


In [ ]:
result.plot(plotter, height=400)


### 3D PPP

In [ ]:
from cameo.flux_analysis.analysis import phenotypic_phase_plane
from cameo.visualization.plotting.with_plotly import PlotlyPlotter
# options may be passed as arguments here
plotter = PlotlyPlotter() #very important!
model = cameo.load_model('data/yeastGEM_het.xml')
result_3D = phenotypic_phase_plane(model, variables=[model.reactions.R07215, model.reactions.r_1992],
                                   objective=model.reactions.r_2111, points=50)
# notice the required plotter argument!
#result_3D.plot(plotter, height=400)



### dFBA analysis

In [ ]:
import numpy as np
np.array(result_3D)
result_3D.plot(plotter)

In [ ]:
from os.path import dirname, join, pardir

from cobra.io import read_sbml_model

from dfba import DfbaModel, ExchangeFlux, KineticVariable

In [ ]:
path_to_model = join(pardir, "sbml-models", "iJR904.xml.gz")
fba_model = read_sbml_model(path_to_model)
fba_model.solver = "glpk"
dfba_model = DfbaModel(fba_model)